# Export Predictions to CSV (for R)

In [1]:
import os
import matplotlib.pyplot as plt
import yaml
#from ct_classifier/ct_classifier.train import create_dataloader, load_model       # NOTE: since we're using these functions across files, it could make sense to put them in e.g. a "util.py" script.
import torch
import pandas as pd
import numpy as np
from torch import softmax as softmax

/home/magali/miniconda3/envs/cv4e_env_new/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#os.getcwd()
os.chdir('/home/magali/ct_classifier/ct_classifier')

In [3]:
from train import load_model
from train import create_dataloader
from train import validate
from dataset import CTDataset
from model import CustomResNet18


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: You are trying to log string value as a metric. This is not recommended.
COMET INFO: Experiment is live on comet.com https://www.comet.com/magalifr/general/22defb6b628a44db8175676162750d7d



## Parameters

In [7]:
config = '/home/magali/ct_classifier/configs/exp_resnet18_sex.yaml'
split = 'val'
BEST_MODEL_PATH = '/home/magali/ct_classifier/best_models/sex_basic_pad_flip_col_rndPers_norm_200ep_run1_198.pt' #model needs to have a number before the pt
#/home/magali/ct_classifier/best_models/sex_basic_pad_flip_col_rndPers_norm_200ep_run1_198.pt
#BEST_MODEL_PATH = '/home/magali/ct_classifier/best_models/sex_basic_pad_flip_col_RoeDeer_run4_80.pt' #model needs to have a number before the pt

## Load Data

In [8]:
# load config
print(f'Using config "{config}"')
cfg = yaml.safe_load(open(config, 'r'))


# setup entities
dl_val = create_dataloader(cfg, split='val')


Using config "/home/magali/ct_classifier/configs/exp_resnet18_sex.yaml"
val number of images 1540 labels 1540 images covered 1540


In [18]:
# load model
#model = torch.load(BEST_MODEL_PATH)
#model = load_model(cfg)[0] #takes first model, refer to bets model here
#model = load_model(cfg)[BEST_MODEL_PATH]
#BEST_MODEL_PATH = '/home/magali/ct_classifier/best_models/sex_basic_run1_epoch54.pt'
#model_instance, _ = load_model(cfg, model_path=BEST_MODEL_PATH)

In [ ]:
#model_instance, start_epoch = load_model(cfg, model_path=BEST_MODEL_PATH)
#model_instance, start_epoch = load_model(cfg, model_path=BEST_MODEL_PATH)



trouble shooting loading model

inspect state dict

In [9]:
saved_state = torch.load(BEST_MODEL_PATH)
print(saved_state.keys())

dict_keys(['loss_train', 'loss_val', 'oa_train', 'oa_val', 'precision', 'recall', 'average precision', 'model'])


In [10]:
#model = CTDataset()  # Replace with your actual model class
model_instance = CustomResNet18(cfg['num_classes'])
print(model_instance.state_dict().keys())

odict_keys(['feature_extractor.conv1.weight', 'feature_extractor.bn1.weight', 'feature_extractor.bn1.bias', 'feature_extractor.bn1.running_mean', 'feature_extractor.bn1.running_var', 'feature_extractor.bn1.num_batches_tracked', 'feature_extractor.layer1.0.conv1.weight', 'feature_extractor.layer1.0.bn1.weight', 'feature_extractor.layer1.0.bn1.bias', 'feature_extractor.layer1.0.bn1.running_mean', 'feature_extractor.layer1.0.bn1.running_var', 'feature_extractor.layer1.0.bn1.num_batches_tracked', 'feature_extractor.layer1.0.conv2.weight', 'feature_extractor.layer1.0.bn2.weight', 'feature_extractor.layer1.0.bn2.bias', 'feature_extractor.layer1.0.bn2.running_mean', 'feature_extractor.layer1.0.bn2.running_var', 'feature_extractor.layer1.0.bn2.num_batches_tracked', 'feature_extractor.layer1.1.conv1.weight', 'feature_extractor.layer1.1.bn1.weight', 'feature_extractor.layer1.1.bn1.bias', 'feature_extractor.layer1.1.bn1.running_mean', 'feature_extractor.layer1.1.bn1.running_var', 'feature_extract

In [11]:
# Loading the full state dict
checkpoint = torch.load(BEST_MODEL_PATH)
# Extracting only the model weights
model_weights = checkpoint['model']
# Initialize your model instance
model_instance = CustomResNet18(cfg['num_classes'])
# Load the model weights
model_instance.load_state_dict(model_weights)

<All keys matched successfully>

In [ ]:
#model_instance, start_epoch = load_model(cfg, model_path=BEST_MODEL_PATH)

#state = torch.load(BEST_MODEL_PATH, map_location='cpu')
#model_instance.load_state_dict(state)

## Visualize

This is up to you to figure out now. :)

In [12]:
print(type(model_instance))
#print(type(model))
#print(model)
#print(prediction)

<class 'model.CustomResNet18'>


In [13]:
#device = cfg['device']
#model.to(device)

model_instance.eval()
#model.eval()

CustomResNet18(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [14]:
all_pred_labels = []
all_labels = []
all_img_path = []

def tuple_to_list(obj):
    if isinstance(obj, tuple):
        return [tuple_to_list(item) for item in obj]
    return obj

with torch.no_grad():               # don't calculate intermediate gradient steps: we don't need them, so this saves memory and is faster
        for idx, (data, labels, img_path) in enumerate(dl_val):

            # put data and labels on device
            #data, labels = data.to(device), labels.to(device)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


            print(data.shape)
            #print(model.classifier.device)
            print(data.device)
            print(device)
            # forward pass
            prediction = model_instance(data)
            #print(prediction[0])
            #print(img_path[0])

            # If you want to track labels and paths as well, you can continue to do that
            pred_label = prediction
            all_pred_labels = all_pred_labels + pred_label.cpu().tolist()
            print(all_pred_labels)
            all_labels = all_labels + labels.tolist()
            print(all_labels)

            # Convert tuple to numpy array
            #img_path_arr = np.array(img_path)
            #img_path_lst = tuple_to_list(img_path)

            #all_img_path = all_img_path + img_path_arr.tolist()

            # Flatten the img_path tuple
            img_path_lst = tuple_to_list(img_path)
            all_img_path.extend(img_path_lst)

            # Print sizes for debugging
            print(f"Iteration {idx}: all_pred_labels size: {len(all_pred_labels)}, all_labels size: {len(all_labels)}, all_img_path size: {len(all_img_path)}")

            
        # After the loop, print or return the summed predictions
        print(all_pred_labels, all_labels, all_img_path)




torch.Size([128, 3, 224, 224])
cpu
cuda
[[2.06581449508667, 0.19587063789367676], [1.3735612630844116, -1.1968884468078613], [3.057605266571045, -1.2725694179534912], [2.506025791168213, -0.078874371945858], [2.5716090202331543, -0.21046924591064453], [0.5267477631568909, 1.2321100234985352], [2.375411033630371, -1.674724817276001], [2.253976583480835, -0.026589922606945038], [2.848470687866211, -1.4145758152008057], [0.4218690097332001, -0.4750967025756836], [-1.951065182685852, 2.4316396713256836], [-1.8778783082962036, 2.3534741401672363], [-0.8158581852912903, 1.7608262300491333], [0.022501427680253983, -0.16710340976715088], [0.46844354271888733, -0.8486748933792114], [-1.5753493309020996, 1.089971899986267], [0.8712472319602966, -0.5337846279144287], [0.2532303035259247, -0.3568282127380371], [1.2697694301605225, -1.494004487991333], [0.14192037284374237, 0.8723667860031128], [0.57486891746521, -0.9617294073104858], [0.19763340055942535, -0.5794380903244019], [0.37998995184898376

In [15]:
print(all_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 

Create dataframes with predictions, labels and image id

In [16]:
label_df= pd.DataFrame(all_labels, columns=['Labels'])
pred_df = pd.DataFrame(all_pred_labels, columns=['pred_female', 'pred_male'])
img_id_df = pd.DataFrame(all_img_path, columns=['Image path'])


In [17]:
print(pred_df)

      pred_female  pred_male
0        2.065814   0.195871
1        1.373561  -1.196888
2        3.057605  -1.272569
3        2.506026  -0.078874
4        2.571609  -0.210469
...           ...        ...
1535     0.235516  -0.170726
1536     0.867555  -1.064979
1537     1.848471  -1.002658
1538     0.549363  -0.289163
1539     0.715841  -0.160033

[1540 rows x 2 columns]


In [18]:
print(img_id_df)

                                             Image path
0     /home/magali/CV4Ecology-summer-school/Prototyp...
1     /home/magali/CV4Ecology-summer-school/Prototyp...
2     /home/magali/CV4Ecology-summer-school/Prototyp...
3     /home/magali/CV4Ecology-summer-school/Prototyp...
4     /home/magali/CV4Ecology-summer-school/Prototyp...
...                                                 ...
1535  /home/magali/CV4Ecology-summer-school/Prototyp...
1536  /home/magali/CV4Ecology-summer-school/Prototyp...
1537  /home/magali/CV4Ecology-summer-school/Prototyp...
1538  /home/magali/CV4Ecology-summer-school/Prototyp...
1539  /home/magali/CV4Ecology-summer-school/Prototyp...

[1540 rows x 1 columns]


In [19]:
result = pd.concat([label_df, pred_df, img_id_df], axis=1, ignore_index=False)
print(result)

      Labels  pred_female  pred_male  \
0          0     2.065814   0.195871   
1          0     1.373561  -1.196888   
2          0     3.057605  -1.272569   
3          0     2.506026  -0.078874   
4          0     2.571609  -0.210469   
...      ...          ...        ...   
1535       0     0.235516  -0.170726   
1536       0     0.867555  -1.064979   
1537       0     1.848471  -1.002658   
1538       0     0.549363  -0.289163   
1539       0     0.715841  -0.160033   

                                             Image path  
0     /home/magali/CV4Ecology-summer-school/Prototyp...  
1     /home/magali/CV4Ecology-summer-school/Prototyp...  
2     /home/magali/CV4Ecology-summer-school/Prototyp...  
3     /home/magali/CV4Ecology-summer-school/Prototyp...  
4     /home/magali/CV4Ecology-summer-school/Prototyp...  
...                                                 ...  
1535  /home/magali/CV4Ecology-summer-school/Prototyp...  
1536  /home/magali/CV4Ecology-summer-school/Prototyp...

convert predictions to with softmax

In [20]:
# Define a function to apply torch.softmax to a list of scores
#def apply_softmax(scores):
#    tensor_scores = torch.tensor(scores)
#    softmax_scores = torch.softmax(tensor_scores, dim=0).tolist()  # Using dim=0 for 1D tensor
#    return softmax_scores

# Apply the function to the 'prediction' column
#result['pred_female_softmax'] = result['pred_female'].apply(torch.softmax)
#result['pred_male_softmax'] = result['pred_male'].apply(apply_softmax)

# Convert DataFrame columns to a tensor
tensor_predictions = torch.tensor(result[['pred_female', 'pred_male']].values)

# Compute softmax along the last dimension (across columns for each row)
softmax_tensor = torch.softmax(tensor_predictions, dim=1)

# Convert the tensor back to a DataFrame
df_softmax = pd.DataFrame(softmax_tensor.numpy(), columns=['pred_female_softmax', 'pred_male_softmax'])

# Concatenate the two DataFrames along axis=1
result = pd.concat([result, df_softmax], axis=1)

print(result)


      Labels  pred_female  pred_male  \
0          0     2.065814   0.195871   
1          0     1.373561  -1.196888   
2          0     3.057605  -1.272569   
3          0     2.506026  -0.078874   
4          0     2.571609  -0.210469   
...      ...          ...        ...   
1535       0     0.235516  -0.170726   
1536       0     0.867555  -1.064979   
1537       0     1.848471  -1.002658   
1538       0     0.549363  -0.289163   
1539       0     0.715841  -0.160033   

                                             Image path  pred_female_softmax  \
0     /home/magali/CV4Ecology-summer-school/Prototyp...             0.866452   
1     /home/magali/CV4Ecology-summer-school/Prototyp...             0.928935   
2     /home/magali/CV4Ecology-summer-school/Prototyp...             0.987006   
3     /home/magali/CV4Ecology-summer-school/Prototyp...             0.929883   
4     /home/magali/CV4Ecology-summer-school/Prototyp...             0.941700   
...                                    

In [21]:
print(result)

      Labels  pred_female  pred_male  \
0          0     2.065814   0.195871   
1          0     1.373561  -1.196888   
2          0     3.057605  -1.272569   
3          0     2.506026  -0.078874   
4          0     2.571609  -0.210469   
...      ...          ...        ...   
1535       0     0.235516  -0.170726   
1536       0     0.867555  -1.064979   
1537       0     1.848471  -1.002658   
1538       0     0.549363  -0.289163   
1539       0     0.715841  -0.160033   

                                             Image path  pred_female_softmax  \
0     /home/magali/CV4Ecology-summer-school/Prototyp...             0.866452   
1     /home/magali/CV4Ecology-summer-school/Prototyp...             0.928935   
2     /home/magali/CV4Ecology-summer-school/Prototyp...             0.987006   
3     /home/magali/CV4Ecology-summer-school/Prototyp...             0.929883   
4     /home/magali/CV4Ecology-summer-school/Prototyp...             0.941700   
...                                    

In [22]:
result.to_csv('../predictions/predictions_sex_aug_bestModel.csv', index=True)
